# Наивный Байес

### Пример 1. (Спам в СМС)

Например, событие А заключается в том, что сообщение содержит "подозрительное слово" (розыгрыш, поздравляем, 1000000$, итд).
Как же сделать так, чтобы телефон сам понимал: это сообщение спам и его вообще можно было нам даже не показывать или же мы правда что то выиграли🙃

Примем, что всё пространство событий(элементарным исходом явлется конкретное СМС) разделено на два кластера: 
$$B_1 - \text{спам},\ \ \  B_2 - \text{не спам}$$

Тогда, благодаря теореме Байесса мы имеем возможность сравнить какое из событий:

$$ (A|B_1) - \text{сообщение содержит "нехорошее" слово и при этом спам}$$
$$ (A|B_2) - \text{сообщение содержит "нехорошее" слово и при этом НЕ спам}$$

Хотя понятно, что чисто логически эти события не имеют смысла, так как B очевиводно произошло раньше события A.

Реализация детектора спама: https://rukovodstvo.net/posts/id_1258/

# Средствами Python

In [1]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=e75b32db913e3f2813dd80b05608834ee63c23e4c9cfc34e9cad9f566c2a33ca
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [2]:
import pandas as pd 
import os # work with filesystem
import re 
import math
import numpy as np
import wget        # library for URL-downloads
import nltk
import csv

import numpy as np
import pandas as pd

In [3]:
data_status = os.path.exists('./SMSSpamCollection')

if data_status == False:
    url = 'https://drive.google.com/uc?export=view&id=1doec_2oDEsHKJRpaLlSiU4ZW1ICUVoRz'
    wget.download(url, 'SMSSpamCollection')

In [4]:
df = pd.read_table('SMSSpamCollection', sep='\t', header=None, names=['label', 'message']) 

In [5]:
# немного обработки

#df['label'] = df.label.map({'ham': 0, 'spam': 1}) 
df['message'] = df.message.map(lambda x: x.lower())
df['message'] = df.message.str.replace('[^\\w\\s]', '') 

<ipython-input-5-4abc327003cc>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['message'] = df.message.str.replace('[^\\w\\s]', '')


In [6]:
#Парсинг на слова
#from nltk.tokenize import word_tokenize
#df['message'] = df['message'].apply(word_tokenize) 

In [7]:
#Определение однокоренных слов

#from nltk.stem import PorterStemmer 
#stemmer = PorterStemmer() 
#df['message'] = df['message'].apply(lambda x: [stemmer.stem(y) for y in x]) 

In [8]:
df

,label,message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will ü b going to esplanade fr home
5569,ham,pity was in mood for that soany other suggest...
5570,ham,the guy did some bitching but i acted like id ...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
counts = count_vect.fit_transform(df['message'])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(counts, df['label'], test_size=0.3, random_state=42) 

In [11]:
from sklearn.naive_bayes import MultinomialNB 
model = MultinomialNB().fit(X_train, y_train)

In [12]:
predicted = model.predict(X_test) 
print(np.mean(predicted == y_test))

0.9808612440191388


# Руками

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.3, random_state=42) 

In [14]:
df_train = pd.concat([X_train.to_frame(), y_train.to_frame()], axis=1)
df_train

,message,label
708,quite late lar ard 12 anyway i wun b drivin,ham
4338,on a tuesday night r u 4 real,ham
5029,go chase after her and run her over while shes...,ham
4921,g says you never answer your texts confirmdeny,ham
2592,still work going onit is very small house,ham
...,...,...
3772,hi wlcome back did wonder if you got eaten by ...,ham
5191,sorry ill call later,ham
5226,prabhaim sorydarealyfrm heart im sory,ham
5390,nt joking seriously i told,ham


In [15]:
df_test = pd.concat([X_test.to_frame(), y_test.to_frame()], axis=1)
df_test

,message,label
3245,squeeeeeze this is christmas hug if u lik my f...,ham
944,and also ive sorta blown him off a couple time...,ham
1044,mmm thats better now i got a roast down me id ...,ham
2484,mm have some kanji dont eat anything heavy ok,ham
812,so theres a ring that comes with the guys cost...,ham
...,...,...
2505,hello my boytoy i made it home and my constant...,ham
2525,free entry into our 250 weekly comp just send ...,spam
4975,aiyo u so poor thing then u dun wan 2 eat u ba...,ham
650,you have won 1000 cash or a 2000 prize to clai...,spam


In [16]:
pd.DataFrame(df_train[df_train['label'] == 'spam']['message'].values.flatten()).value_counts()

loan for any purpose 500  75000 homeowners  tenants welcome have you been previously refused we can still help call free 0800 1956669 or text back help        3
free for 1st week no1 nokia tone 4 ur mob every week just txt nokia to 8007 get txting and tell ur mates wwwgetzedcouk pobox 36504 w45wq norm150ptone 16       3
bored housewives chat n date now 08717507711 btnational rate 10pmin only from landlines                                                                        2
for ur chance to win a 250 cash every wk txt action to 80608 tscs wwwmovietriviatv custcare 08712405022 1x150pwk                                               2
you are chosen to receive a 350 award pls call claim number 09066364311 to collect your award which you are selected to receive as a valued mobile customer    2
                                                                                                                                                              ..
good luck draw takes place 28th fe

In [17]:
inapropriate_words_dict = {'sex' : 0, 'sexy' : 0, 'win' : 0, 'free' : 0, 'gift' : 0, 'porn' : 0, 'chosen' : 0, 'chance' : 0}
inapropriate_words_list = inapropriate_words_dict.keys()

In [18]:
for word in inapropriate_words_list:
  df_train[word] = df_train['message'].str.contains(word)
df_train

,message,label,sex,sexy,win,free,gift,porn,chosen,chance
708,quite late lar ard 12 anyway i wun b drivin,ham,False,False,False,False,False,False,False,False
4338,on a tuesday night r u 4 real,ham,False,False,False,False,False,False,False,False
5029,go chase after her and run her over while shes...,ham,False,False,False,False,False,False,False,False
4921,g says you never answer your texts confirmdeny,ham,False,False,False,False,False,False,False,False
2592,still work going onit is very small house,ham,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
3772,hi wlcome back did wonder if you got eaten by ...,ham,False,False,False,False,False,False,False,False
5191,sorry ill call later,ham,False,False,False,False,False,False,False,False
5226,prabhaim sorydarealyfrm heart im sory,ham,False,False,False,False,False,False,False,False
5390,nt joking seriously i told,ham,False,False,False,False,False,False,False,False


In [19]:
df_spam_train = df_train[df_train['label'] == 'spam']
df_spam_train

,message,label,sex,sexy,win,free,gift,porn,chosen,chance
1780,loan for any purpose 500 75000 homeowners te...,spam,False,False,False,True,False,False,False,False
4104,u have a secret admirer who is looking 2 make ...,spam,False,False,False,False,False,False,False,False
4012,please call our customer service representativ...,spam,False,False,False,True,False,False,False,False
1765,hi 07734396839 ibh customer loyalty offer the ...,spam,False,False,False,False,False,False,False,False
789,5 free top polyphonic tones call 087018728737 ...,spam,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
3005,youve won tkts to the euro2004 cup final or 80...,spam,False,False,False,False,False,False,False,False
2558,this message is brought to you by gmw ltd and ...,spam,False,False,False,False,False,False,False,False
474,want 2 get laid tonight want real dogging loca...,spam,False,False,False,False,False,False,False,False
3385,free msgwe billed your mobile number by mistak...,spam,False,False,False,True,False,False,False,False


Расчет условной вероятности того, что среди сообщений с неприемлемым словом будет сообщение со спамом.

In [20]:
for word in df_train.columns[2:]:
  inapropriate_words_dict[word] = len(df_spam_train[df_spam_train[word] == True]) / len(df_train[df_train[word] == True])

In [21]:
inapropriate_words_dict

{'sex': 0.6785714285714286,
 'sexy': 0.5,
 'win': 0.6166666666666667,
 'free': 0.7538461538461538,
 'gift': 0.7272727272727273,
 'porn': 0.5,
 'chosen': 1.0,
 'chance': 0.71875}

In [22]:
for word in inapropriate_words_list:
  df_test[word] = df_test['message'].str.contains(word)
df_test

,message,label,sex,sexy,win,free,gift,porn,chosen,chance
3245,squeeeeeze this is christmas hug if u lik my f...,ham,False,False,False,False,False,False,False,False
944,and also ive sorta blown him off a couple time...,ham,False,False,False,False,False,False,False,False
1044,mmm thats better now i got a roast down me id ...,ham,False,False,False,False,False,False,False,False
2484,mm have some kanji dont eat anything heavy ok,ham,False,False,False,False,False,False,False,False
812,so theres a ring that comes with the guys cost...,ham,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
2505,hello my boytoy i made it home and my constant...,ham,False,False,False,False,False,False,False,False
2525,free entry into our 250 weekly comp just send ...,spam,False,False,True,True,False,False,False,False
4975,aiyo u so poor thing then u dun wan 2 eat u ba...,ham,False,False,False,False,False,False,False,False
650,you have won 1000 cash or a 2000 prize to clai...,spam,False,False,False,False,False,False,False,False


In [23]:
df_test['prediction'] = False

In [24]:
for word in inapropriate_words_list: 
  df_test['prediction'] |= np.where((df_test[word] == True) & (inapropriate_words_dict[word] > 0.5), True, False)

df_test['prediction'] = df_test['prediction'].replace({True: 'spam', False: 'ham'})
df_test

,message,label,sex,sexy,win,free,gift,porn,chosen,chance,prediction
3245,squeeeeeze this is christmas hug if u lik my f...,ham,False,False,False,False,False,False,False,False,ham
944,and also ive sorta blown him off a couple time...,ham,False,False,False,False,False,False,False,False,ham
1044,mmm thats better now i got a roast down me id ...,ham,False,False,False,False,False,False,False,False,ham
2484,mm have some kanji dont eat anything heavy ok,ham,False,False,False,False,False,False,False,False,ham
812,so theres a ring that comes with the guys cost...,ham,False,False,False,False,True,False,False,False,spam
...,...,...,...,...,...,...,...,...,...,...,...
2505,hello my boytoy i made it home and my constant...,ham,False,False,False,False,False,False,False,False,ham
2525,free entry into our 250 weekly comp just send ...,spam,False,False,True,True,False,False,False,False,spam
4975,aiyo u so poor thing then u dun wan 2 eat u ba...,ham,False,False,False,False,False,False,False,False,ham
650,you have won 1000 cash or a 2000 prize to clai...,spam,False,False,False,False,False,False,False,False,ham
